In [13]:
import pandas as pd ## biblioteca de estruturação e analise de dados
import numpy as np ## biblioteca de algebra linear entre outras utilidades

import random ## biblioteca de geracao de numeros aleatorios
import datetime ## biblioteca para criar/trabalhar datas

from tqdm import tqdm ## loading bar

'''
Do enunciado:

"Todas as informações de vendas, 
tais como o valor da venda e qual empresa efetuou a compra,
estão armazenadas em um banco de dados PostgreSQL."

Para poder simular uma analise simples, iremos gerar dados aleatorios.
A arquitetura do banco é simplificada ao maximo, 
a uma unica entidade composta apenas de 3 atributos.

Essa arquitetura nao corresponde a uma esquematica ideal, 
é apenas uma simplificacao para fazer uma poc do exercicio.

Um banco ideal deveria contemplar mais informacoes sobre as vendas,
como informacao de cada produto, preço individual, possiveis descontos/promocoes,
e outras informacoes pertinentes. O mesmo deveria estar devidamente normalizado,
de maneira a conter diversas tabelas, que poderiam ser exportadas em arquivos csv
separados, ou ainda, "merged" em uma unica tabela com as informacoes relevantes
para a analise.

-----------------------------------------------------

O equivalente é valido para o caso dos dados do hubspot.

'''

## definindo sementes para replicabilidade
random.seed(2005) 
np.random.seed(2005)

##-------------------------------------------------------------##
## criando o primeiro arquivo csv proveniente do banco postgresql
##-------------------------------------------------------------##

df = pd.DataFrame([], columns=['id_venda','cnpj','data','valor']) ## criando dataframe em branco

min_date = pd.to_datetime('2017-01-01').date() ## definindo data minima, iremos contemplar apenas o ano de 2017

doy_today = datetime.datetime.now().date().timetuple().tm_yday

id_venda = 0

for i in tqdm(np.arange(1,101,1)): ## gerando 100 entradas unicas de empresas
    
    number_of_purchases = random.randint(1,20) ## definindo um numero aleatorio de compras que a empresa efetuou
    
    cnpj = random.randint(1*10**14, 9*10**14) ## definindo o cnpj ficticio da empresa
    
    for j in np.arange(number_of_purchases): ## para cada compra, definindo seu valor e data
        
        value = random.uniform(10,200)
        
        date = (min_date + datetime.timedelta(days=random.randint(0,doy_today-1)))
        
        df.loc[len(df)] = [id_venda, cnpj, date, value] ## anexando a informacao ao dataframe
        
        id_venda = id_venda + 1

##------------------------------------------------------------##
## criando o segundo arquivo csv, proveniente do serviço HubSpot
##------------------------------------------------------------##

cnpjs = df.cnpj.unique() ## salvando os cnpjs das empresas
id_representantes = np.random.randint(100,1000,25) ## gerando até 25 representantes diferentes

df2 = pd.DataFrame([], columns=['cnpj','id_representante','id_contrato','data_inicio']) ## criando dataframe em branco


for cnpj in tqdm(cnpjs): ## percorrendo as empresas
    
    representante = random.choice(id_representantes) ## escolhendo um representante para aquela empresa
    
    number_contracts = np.random.randint(1,4) ## definindo qtos contratos aquele representante fechou (1-3)
    
    temp_df = df.loc[df.cnpj==cnpj] ## fazendo um dataframe dummy para apenas aquela empresa
    
    for contract in np.arange(number_contracts): ## percorrendo os contratos
    
        if str(contract).endswith('0'): ## se for o primeiro, cria uma data anterior a primeira compra
    
            contract_id = str(representante)+'_'+str(contract)

            max_date = np.min(temp_df.data)
            
            try:
            
                date = max_date - datetime.timedelta(days=np.random.randint(0,(max_date.timetuple().tm_yday-1)))
                
            except:
                
                date = max_date
            
        else: ## caso contrario, cria uma data posterior a primeira compra
            
            contract_id = str(representante)+'_'+str(contract)

            min_date = np.min(temp_df.data)
            
            date = min_date + datetime.timedelta(days=np.random.randint(0, doy_today-min_date.timetuple().tm_yday))
            
        df2.loc[len(df2)] = [cnpj, representante, contract_id, date] # armazena os dados

##------------------------------------------------------##
## criando um terceiro df, para relacionar os 2 anteriores
##------------------------------------------------------##

df3 = pd.DataFrame([], columns=['id_contrato','id_venda'])

for index in tqdm(df.index):
    
    dummy = df.loc[index]
    
    id_venda = dummy.id_venda
    data = dummy['data']
    cnpj = dummy.cnpj
    
    dummy_2 = df2.loc[(df2.cnpj==cnpj)&(df2.data_inicio<=data)]
    
    ids_contrato = dummy_2.id_contrato.get_values()
    id_contrato = np.random.choice(ids_contrato)
    
    df3.loc[len(df3)] = [id_contrato, id_venda]
    
df = pd.merge(df,df3)

df.to_csv('UauOffice_Vendas.csv', index=False, encoding='latin-1') ## salvando o dataset gerado
df2.to_csv('hubspot_Vendas.csv', index=False, encoding='latin-1') ## salvando o dataset gerado

100%|██████████| 918/918 [00:02<00:00, 374.37it/s]
